In [1]:
import garner

In [2]:
garner.login("kipgparker@gmail.com", "d9a49411-b7c2-42bd-bb43-16f4bdaccb40")

Enter Password


Password: ·········


Authenticated


In [3]:
query = """query ListPools(
    $filter: ModelPoolFilterInput
    $limit: Int
    $nextToken: String
  ) {
    listPools(filter: $filter, limit: $limit, nextToken: $nextToken) {
      items {
        id
        title
        privateKey
      }
    }
  }"""

params = {
    "filter": {"privateKey" : {"eq": garner.auth.private_key}}
}


response = garner.api.execute_gql(query, params)

In [10]:
print("working on: {}".format('\033[1m' +output['data']['listPools']['items'][0]['title']))

In [5]:
garner.storage.upload_file('movie.gif')

In [14]:
b = 0
def bruh(b):
    b += 1
    return True

if(bruh(b)):
    print(b)

0


In [12]:
if(bruh):
    print(b)

0
